This script takes as input the LSTM or RNN weights found by train.py
change the path in line 176 of this script to point to the h5 file
with LSTM or RNN weights generated by train.py

Author: Niek Tax

In [1]:
from __future__ import division
import csv
import copy
import numpy as np
import distance
from itertools import izip, islice
import unicodecsv
from sklearn import metrics
from math import sqrt
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from jellyfish._jellyfish import damerau_levenshtein_distance
from collections import Counter

In [2]:
from keras.models import load_model
from theano.ifelse import ifelse #added this

Using Theano backend.


# Read 1

In [3]:
eventlog = "helpdesk.csv"
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []
timeseqs2 = []
times = []
times2 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [4]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161

In [5]:
for row in islice(spamreader, 0, 174):
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:        
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
        line = ''
        times = []
        numlines+=1
    line+=unichr(int(row[1])+ascii_offset)
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    times.append(timediff)
    times2.append(timediff2)
    lasteventtime = t
    firstLine = False

# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
numlines+=1

In [6]:
divisor = np.mean([item for sublist in timeseqs for item in sublist])
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist])
print('divisor2: {}'.format(divisor2))

divisor: 209169.482759
divisor2: 399919.678161


In [7]:
elems_per_fold = int(round(numlines/3))
fold1 = lines[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]

fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]

lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2

In [8]:
step = 1
sentences = []
softness = 0
next_chars = []
lines = map(lambda x: x+'!',lines)
maxlen = max(map(lambda x: len(x),lines))

chars = map(lambda x : set(x),lines)
chars = list(set().union(*chars))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

total chars: 5, target chars: 6
{0: u'\xa2', 1: u'\xa4', 2: u'\xa7', 3: u'\xa9', 4: u'\xaa'}


# Read 2

In [9]:
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []  # relative time since previous event
timeseqs2 = [] # relative time since case start
timeseqs3 = [] # absolute time of previous event
times = []
times2 = []
times3 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [10]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers

['CaseID', 'ActivityID', 'CompleteTimestamp']

In [11]:
for row in islice(spamreader, 0, 174):
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:        
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
        line = ''
        times = []
        times2 = [] #added this
        times3 = [] #added this
        numlines+=1
    line+=unichr(int(row[1])+ascii_offset)
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    #timediff = log(timediff+1)
    times.append(timediff)
    times2.append(timediff2)
    times3.append(datetime.fromtimestamp(time.mktime(t)))
    lasteventtime = t
    firstLine = False

# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
numlines+=1

In [12]:
numlines, len(lines), len(timeseqs), len(timeseqs2), len(timeseqs3)

(51, 50, 50, 50, 50)

In [13]:
len(timeseqs3[1])

3

In [14]:
timeseqs3

[[datetime.datetime(2012, 4, 3, 16, 55, 38),
  datetime.datetime(2012, 4, 3, 16, 55, 53),
  datetime.datetime(2012, 4, 5, 17, 15, 52)],
 [datetime.datetime(2010, 10, 29, 18, 14, 6),
  datetime.datetime(2010, 11, 4, 1, 16, 11),
  datetime.datetime(2010, 11, 4, 1, 21, 17)],
 [datetime.datetime(2010, 12, 15, 23, 31, 53),
  datetime.datetime(2010, 12, 16, 17, 1, 7),
  datetime.datetime(2010, 12, 16, 17, 8, 19)],
 [datetime.datetime(2012, 4, 3, 21, 8, 32),
  datetime.datetime(2012, 4, 3, 21, 45, 33),
  datetime.datetime(2012, 4, 3, 21, 47, 22),
  datetime.datetime(2012, 4, 3, 22, 15, 2),
  datetime.datetime(2012, 4, 4, 0, 7, 28)],
 [datetime.datetime(2011, 10, 25, 23, 13, 58),
  datetime.datetime(2011, 10, 25, 23, 58, 55),
  datetime.datetime(2011, 10, 26, 17, 54, 46)],
 [datetime.datetime(2012, 7, 5, 16, 30, 32),
  datetime.datetime(2012, 7, 5, 17, 25, 45),
  datetime.datetime(2012, 7, 5, 17, 44, 46)],
 [datetime.datetime(2010, 5, 7, 21, 2, 21),
  datetime.datetime(2010, 5, 7, 21, 2, 34),


In [15]:
len(timeseqs2[0])

3

In [16]:
fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
#fold3_t4 = timeseqs4[2*elems_per_fold:]

lines = fold3
lines_t = fold3_t
lines_t2 = fold3_t2
lines_t3 = fold3_t3
#lines_t4 = fold1_t4 + fold2_t4

In [17]:
elems_per_fold, len(lines), len(lines_t), len(lines_t2), len(lines_t3)

(17, 16, 16, 16, 16)

In [19]:
# set parameters
predict_size = 1

# load model, set this to the model generated by train.py
model = load_model('output_files/models/model_89-1.50.h5')

ValueError: epsilon must be at least 1e-5, got 9.99999974738e-06

In [ ]:
len(timeseqs3[0])

# Helper functions

In [ ]:
# define helper functions
def encode(sentence, times, times3, maxlen=maxlen):
    num_features = len(chars)+5
    X = np.zeros((1, maxlen, num_features), dtype=np.float32)
    leftpad = maxlen-len(sentence)
    times2 = np.cumsum(times)
    for t, char in enumerate(sentence):
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        timesincemidnight = times3[t]-midnight
        multiset_abstraction = Counter(sentence[:t+1])
        for c in chars:
            if c==char:
                X[0, t+leftpad, char_indices[c]] = 1
        X[0, t+leftpad, len(chars)] = t+1
        X[0, t+leftpad, len(chars)+1] = times[t]/divisor
        X[0, t+leftpad, len(chars)+2] = times2[t]/divisor2
        X[0, t+leftpad, len(chars)+3] = timesincemidnight.seconds/86400
        X[0, t+leftpad, len(chars)+4] = times3[t].weekday()/7
    return X

In [ ]:
def getSymbol(predictions):
    maxPrediction = 0
    symbol = ''
    i = 0;
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_char[i]
        i += 1
    return symbol

# Prediction

In [ ]:
one_ahead_gt = []
one_ahead_pred = []

two_ahead_gt = []
two_ahead_pred = []

three_ahead_gt = []
three_ahead_pred = []

In [ ]:
# make predictions
with open('output_files/results/next_activity_and_time_%s' % eventlog, 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Prefix length", "Groud truth", "Predicted", "Levenshtein", "Damerau", "Jaccard", "Ground truth times", "Predicted times", "RMSE", "MAE", "Median AE"])
    for prefix_size in range(2,maxlen):
        print(prefix_size)
        for line, times, times3 in izip(lines, lines_t, lines_t3):
            times.append(0)
            cropped_line = ''.join(line[:prefix_size])
            cropped_times = times[:prefix_size]
            cropped_times3 = times3[:prefix_size]
            if '!' in cropped_line:
                continue # make no prediction for this case, since this case has ended already
            ground_truth = ''.join(line[prefix_size:prefix_size+predict_size])
            ground_truth_t = times[prefix_size:prefix_size+predict_size]
            
            predicted = ''
            predicted_t = []
            for i in range(predict_size):
                if len(ground_truth)<=i:
                    continue
                enc = encode(cropped_line, cropped_times, cropped_times3)
                y = model.predict(enc, verbose=0)
                y_char = y[0][0]
                y_t = y[1][0][0]
                
                prediction = getSymbol(y_char)              
                cropped_line += prediction
                if y_t<0:
                    y_t=0
                cropped_times.append(y_t)
                y_t = y_t * divisor
                cropped_times3.append(cropped_times3[-1] + timedelta(seconds=y_t))
                predicted_t.append(y_t)
                if i==0:
                    if len(ground_truth_t)>0:
                        one_ahead_pred.append(y_t)
                        one_ahead_gt.append(ground_truth_t[0])
                if i==1:
                    if len(ground_truth_t)>1:
                        two_ahead_pred.append(y_t)
                        two_ahead_gt.append(ground_truth_t[1])
                if i==2:
                    if len(ground_truth_t)>2:
                        three_ahead_pred.append(y_t)
                        three_ahead_gt.append(ground_truth_t[2])
                if prediction == '!': # end of case was just predicted, therefore, stop predicting further into the future
                    print('! predicted, end case')
                    break
                predicted += prediction
            output = []
            if len(ground_truth)>0:
                output.append(prefix_size)
                output.append(unicode(ground_truth).encode("utf-8"))
                output.append(unicode(predicted).encode("utf-8"))
                output.append(1 - distance.nlevenshtein(predicted, ground_truth))
                dls = 1 - (damerau_levenshtein_distance(unicode(predicted), unicode(ground_truth)) / max(len(predicted),len(ground_truth)))
                if dls<0:
                    dls=0 # we encountered problems with Damerau-Levenshtein Similarity on some linux machines where the default character encoding of the operating system caused it to be negative, this should never be the case
                output.append(dls)
                output.append(1 - distance.jaccard(predicted, ground_truth))
                output.append('; '.join(str(x) for x in ground_truth_t))
                output.append('; '.join(str(x) for x in predicted_t))
                if len(predicted_t)>len(ground_truth_t): # if predicted more events than length of case, only use needed number of events for time evaluation
                    predicted_t = predicted_t[:len(ground_truth_t)]
                if len(ground_truth_t)>len(predicted_t): # if predicted less events than length of case, put 0 as placeholder prediction
                    predicted_t.extend(range(len(ground_truth_t)-len(predicted_t)))
                if len(ground_truth_t)>0 and len(predicted_t)>0:
                    output.append('')
                    output.append(metrics.mean_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                    output.append(metrics.median_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                else:
                    output.append('')
                    output.append('')
                    output.append('')
                spamwriter.writerow(output)

In [20]:
maxlen=9
for prefix_size in range(2,maxlen):
    print(prefix_size)
    for line, times, times3 in izip(lines, lines_t, lines_t3):
        #print('line: {0}'.format(line))
        print('times: {0}, len: {1}'.format(times, len(times)))
        print('times3: {0}, len: {1}'.format(times3, len(times3)))
        times.append(0)
    
        cropped_line = ''.join(line[:prefix_size])
        cropped_times = times[:prefix_size]
        cropped_times3 = times3[:prefix_size]
        
        #print('cropped_line: {0}'.format(cropped_line))
        print('cropped_times: {0}'.format(cropped_times))
        print('cropped_times3: {0}'.format(cropped_times3))   
        
        if '!' in cropped_line:
            continue # make no prediction for this case, since this case has ended already
            
        ground_truth = ''.join(line[prefix_size:prefix_size+predict_size])
        ground_truth_t = times[prefix_size:prefix_size+predict_size]
        #print('ground_truth: {0}'.format(ground_truth))
        print('ground_truth_t: {0}'.format(ground_truth_t))
        
        predicted = ''
        predicted_t = []
        print('\n')
        enc = encode(cropped_line, cropped_times, cropped_times3)
        print(enc)
    break

2
times: [0, 834, 345587, 265102], len: 4
times3: [datetime.datetime(2011, 11, 25, 23, 33, 54), datetime.datetime(2011, 11, 25, 23, 47, 48), datetime.datetime(2011, 11, 29, 23, 47, 35), datetime.datetime(2011, 12, 3, 1, 25, 57)], len: 4
cropped_times: [0, 834]
cropped_times3: [datetime.datetime(2011, 11, 25, 23, 33, 54), datetime.datetime(2011, 11, 25, 23, 47, 48)]
ground_truth_t: [345587]




NameError: name 'encode' is not defined

In [ ]:
enc = encode(cropped_line, cropped_times, cropped_times3)